In [1]:
!pip install nixtlats


In [3]:
import pandas as pd

In [7]:

df = pd.read_csv("data-8013-trends.csv")
df_sorted = df.sort_values(by='BILLING_DATE')
df.head()

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,BILLING_DATE,SITE,BRAND,FORMAT_DESC,MH_SEGMENT,MH_FAMILY,MH_CLASS,MH_BRICK,ARTICLE,TOTAL_BILLING_QTY,TOTAL_GROSS_MARGIN,TOTAL_MARKDOWN,TOTAL_NET_SALES,TOTAL_GROSS_SALES,TOTAL_DISCOUNT
0,712337,712337,712337,712337,712337,2022-09-30,8013,04017,TRENDS,WOMENS WEAR,WESTERN WEAR,WINTER WEAR,SWEATSHIRTS,441146771010,1,78.66,0.0,334.28,499.0,164.72
1,669408,669408,669408,669408,669408,2022-09-30,8013,D525,TRENDS,WOMENS WEAR,WESTERN WEAR,CASUAL WEAR,T SHIRTS,441137882004,1,71.91,0.0,240.26,399.0,158.74
2,236195,236195,236195,236195,236195,2022-09-30,8013,09888,TRENDS,MENS CASUAL,ACTIVE WEAR,TOPS,T SHIRTS,441134761003,1,88.72,0.0,322.58,599.0,276.42
3,298014,298014,298014,298014,298014,2022-09-30,8013,04063,TRENDS,MENS CASUAL,ACTIVE WEAR,TOPS,T SHIRTS,441145712004,1,408.46,0.0,799.00,799.0,0.00
4,776725,776725,776725,776725,776725,2022-09-30,8013,04042,TRENDS,MENS WEAR,SMART CASUALS,BOTTOMS,TROUSERS,441138083006,1,239.82,0.0,790.98,999.0,208.02


In [8]:
# COMPLETE TIMEGPT IMPLEMENTATION WITH EXOGENOUS VARIABLES - FIXED VERSION
# ========================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from nixtlats import TimeGPT
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Initialize TimeGPT client with your token
timegpt = TimeGPT(token='nixak-AI31YupjpWhin07kPGKOhvW5zj8IliWOlTSWLEInpOuKHNWNhtSETsXIFgyEiYT58g3Hk0hvMFhnJpdS')

# ========================================================================
# STEP 1: EXOGENOUS FEATURE CREATION
# ========================================================================

def create_calendar_features(df):
    """Create comprehensive calendar features for Indian retail"""
    df = df.copy()
    df['BILLING_DATE'] = pd.to_datetime(df['BILLING_DATE'])
    
    # Basic temporal features
    df['year'] = df['BILLING_DATE'].dt.year
    df['month'] = df['BILLING_DATE'].dt.month
    df['day_of_week'] = df['BILLING_DATE'].dt.dayofweek
    df['day_of_year'] = df['BILLING_DATE'].dt.dayofyear
    df['week_of_year'] = df['BILLING_DATE'].dt.isocalendar().week
    df['quarter'] = df['BILLING_DATE'].dt.quarter
    
    # Weekend indicator
    df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
    
    # Month-end/start effects
    df['is_month_start'] = df['BILLING_DATE'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['BILLING_DATE'].dt.is_month_end.astype(int)
    df['is_quarter_start'] = df['BILLING_DATE'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['BILLING_DATE'].dt.is_quarter_end.astype(int)
    
    return df

def create_indian_holiday_features(df):
    """Add Indian holiday indicators based on 2024 calendar"""
    df = df.copy()
    
    # Major Indian holidays for 2024 (extend for other years)
    major_holidays = {
        '2024-01-26': 'republic_day',
        '2024-03-25': 'holi', 
        '2024-04-11': 'eid_ul_fitr',
        '2024-04-17': 'ram_navami',
        '2024-08-15': 'independence_day',
        '2024-08-26': 'janmashtami',
        '2024-10-12': 'dussehra',
        '2024-10-31': 'diwali',
        '2024-12-25': 'christmas'
    }
    
    # Create holiday indicators
    for date_str, holiday_name in major_holidays.items():
        df[f'is_{holiday_name}'] = (df['BILLING_DATE'].dt.strftime('%Y-%m-%d') == date_str).astype(int)
    
    # Holiday proximity features (corrected variable names)
    diwali_date = pd.to_datetime('2024-10-31')
    for days in [1, 2, 3]:
        df[f'days_before_diwali_{days}'] = (df['BILLING_DATE'] == diwali_date - pd.Timedelta(days=days)).astype(int)
        df[f'days_after_diwali_{days}'] = (df['BILLING_DATE'] == diwali_date + pd.Timedelta(days=days)).astype(int)
    
    return df

def create_business_features(df):
    """Create features from existing business data"""
    df = df.copy()
    
    # Categorical encoding for key business dimensions
    categorical_cols = ['SITE', 'BRAND', 'FORMAT_DESC', 'MH_SEGMENT', 'MH_FAMILY']
    
    for col in categorical_cols:
        if col in df.columns:
            # Create frequency encoding
            freq_map = df[col].value_counts(normalize=True).to_dict()
            df[f'{col}_frequency'] = df[col].map(freq_map)
            
            # One-hot encode top categories only (to avoid high dimensionality)
            top_categories = df[col].value_counts().head(5).index
            for category in top_categories:
                df[f'{col}_{category}'] = (df[col] == category).astype(int)
    
    # Price-related features (handle division by zero)
    df['avg_selling_price'] = df['TOTAL_NET_SALES'] / df['TOTAL_BILLING_QTY'].replace(0, 1)
    df['discount_rate'] = df['TOTAL_DISCOUNT'] / df['TOTAL_GROSS_SALES'].replace(0, 1)
    df['margin_rate'] = df['TOTAL_GROSS_MARGIN'] / df['TOTAL_NET_SALES'].replace(0, 1)
    
    return df

def prepare_exogenous_data_for_timegpt(df):
    """Prepare exogenous data in TimeGPT format"""
    print("   • Creating exogenous features...")
    
    # Step 1: Create all features
    df_enhanced = create_calendar_features(df)
    df_enhanced = create_indian_holiday_features(df_enhanced)
    df_enhanced = create_business_features(df_enhanced)
    
    # Step 2: Define predictable features (can be calculated for future dates)
    predictable_features = [
        'year', 'month', 'day_of_week', 'is_weekend', 'is_month_start', 
        'is_month_end', 'is_quarter_start', 'is_quarter_end', 'quarter',
        'is_republic_day', 'is_holi', 'is_diwali', 'is_independence_day',
        'is_christmas'
    ]
    
    # Filter to only include features that actually exist in the data
    available_features = [f for f in predictable_features if f in df_enhanced.columns]
    
    print(f"   • Created {len(available_features)} predictable exogenous features")
    
    return df_enhanced, available_features

# ========================================================================
# STEP 2: DATA CLEANING AND PREPARATION
# ========================================================================

def clean_and_prepare_data_with_exogenous(df_sorted, top_n_products=8):
    """Enhanced data cleaning with exogenous feature integration"""
    print("   • Creating exogenous features before data aggregation...")
    
    # Step 1: Create exogenous features on raw data FIRST
    df_enhanced, predictable_features = prepare_exogenous_data_for_timegpt(df_sorted)
    
    # Handle negative sales values
    print(f"   • Found {(df_enhanced['TOTAL_NET_SALES'] < 0).sum()} negative sales records")
    df_enhanced.loc[df_enhanced['TOTAL_NET_SALES'] < 0, 'TOTAL_NET_SALES'] = 0
    
    # Create PRODUCT_KEY if not exists
    if 'PRODUCT_KEY' not in df_enhanced.columns:
        df_enhanced['PRODUCT_KEY'] = (
            df_enhanced['BRAND'].astype(str) + ' | ' + 
            df_enhanced['MH_SEGMENT'].astype(str) + ' | ' + 
            df_enhanced['MH_FAMILY'].astype(str) + ' | ' + 
            df_enhanced['MH_CLASS'].astype(str) + ' | ' + 
            df_enhanced['MH_BRICK'].astype(str)
        )
    
    # Product filtering
    product_analysis = df_enhanced.groupby('PRODUCT_KEY').agg({
        'TOTAL_NET_SALES': ['sum', 'mean', 'count', 'std'],
        'BILLING_DATE': ['min', 'max', 'nunique'],
        'TOTAL_BILLING_QTY': 'sum',
        'TOTAL_GROSS_MARGIN': 'sum'
    }).round(2)
    
    product_analysis.columns = ['total_sales', 'avg_sales', 'transaction_count','sales_std', 'first_date', 'last_date', 'unique_days','total_qty', 'total_margin']
    
    # Calculate quality metrics
    product_analysis['data_span_days'] = (product_analysis['last_date'] - product_analysis['first_date']).dt.days + 1
    product_analysis['data_density'] = product_analysis['unique_days'] / product_analysis['data_span_days']
    product_analysis['cv'] = product_analysis['sales_std'] / product_analysis['avg_sales'].replace(0, 1)
    
    # Product filtering criteria
    viable_products = product_analysis[
        (product_analysis['unique_days'] >= 60) &
        (product_analysis['data_span_days'] >= 90) &
        (product_analysis['data_density'] >= 0.3) &
        (product_analysis['total_sales'] >= 10000) &
        (product_analysis['transaction_count'] >= 30) &
        (product_analysis['cv'] < 3)
    ].sort_values('total_sales', ascending=False)
    
    print(f"   📊 Product filtering with exogenous features:")
    print(f"      • Viable for forecasting: {len(viable_products)}")
    print(f"      • Selected top: {min(top_n_products, len(viable_products))}")
    
    if len(viable_products) == 0:
        viable_products = product_analysis[
            (product_analysis['unique_days'] >= 30) &
            (product_analysis['total_sales'] >= 5000)
        ].sort_values('total_sales', ascending=False)
    
    selected_products = viable_products.head(min(top_n_products, len(viable_products))).index.tolist()
    filtered_df = df_enhanced[df_enhanced['PRODUCT_KEY'].isin(selected_products)].copy()
    
    print("   • Aggregating with enhanced exogenous variables...")
    
    # Aggregation dictionary
    agg_dict = {
        'TOTAL_NET_SALES': 'sum',
        'TOTAL_BILLING_QTY': 'sum',
        'TOTAL_GROSS_MARGIN': 'sum',
        'TOTAL_GROSS_SALES': 'sum',
        'TOTAL_DISCOUNT': 'sum'
    }
    
    # Add predictable exogenous features to aggregation (using 'first' since they're constant per day)
    for feature in predictable_features:
        if feature in filtered_df.columns:
            agg_dict[feature] = 'first'
    
    daily_sales = filtered_df.groupby(['BILLING_DATE', 'PRODUCT_KEY']).agg(agg_dict).reset_index()
    
    # Rename columns for TimeGPT
    column_mapping = {
        'BILLING_DATE': 'ds',
        'TOTAL_NET_SALES': 'y',
        'PRODUCT_KEY': 'unique_id',
        'TOTAL_BILLING_QTY': 'qty',
        'TOTAL_GROSS_MARGIN': 'margin',
        'TOTAL_GROSS_SALES': 'gross_sales',
        'TOTAL_DISCOUNT': 'discount'
    }
    
    timegpt_data = daily_sales.rename(columns=column_mapping)
    
    # Data type optimization
    timegpt_data['ds'] = pd.to_datetime(timegpt_data['ds'])
    for col in ['y', 'qty', 'margin', 'gross_sales', 'discount']:
        if col in timegpt_data.columns:
            timegpt_data[col] = pd.to_numeric(timegpt_data[col], errors='coerce')
    
    # Remove invalid records
    timegpt_data = timegpt_data.dropna(subset=['y'])
    
    print(f"   ✅ Enhanced data with exogenous features: {len(timegpt_data):,} rows, {timegpt_data['unique_id'].nunique()} products")
    print(f"   📊 Exogenous features included: {len([f for f in predictable_features if f in timegpt_data.columns])}")
    
    return timegpt_data, predictable_features

# ========================================================================
# STEP 3: FREQUENCY FIXING
# ========================================================================

def fix_frequency_issues_complete(data):
    """Robust frequency fixing optimized for irregular data patterns"""
    print("   • Applying robust frequency correction for irregular data...")
    
    cleaned_data = []
    successful_products = []
    
    for product_id, group in data.groupby('unique_id'):
        # Sort and remove duplicates
        group = group.sort_values('ds').drop_duplicates(subset=['ds'], keep='first')
        
        print(f"      Processing: {product_id[:40]}... ({len(group)} records)")
        
        # Skip products with insufficient data
        if len(group) < 50:
            print(f"        ❌ Insufficient data: {len(group)} < 50 days")
            continue
        
        # Create complete date range
        date_range = pd.date_range(
            start=group['ds'].min(),
            end=group['ds'].max(),
            freq='D'
        )
        
        expected_points = len(date_range)
        missing_points = expected_points - len(group)
        missing_ratio = missing_points / expected_points
        
        print(f"        📊 Missing ratio: {missing_ratio:.2%}")
        
        # Skip products with too much missing data
        if missing_ratio > 0.7:
            print(f"        ❌ Too sparse: {missing_ratio:.1%} missing data")
            continue
        
        # Create complete time series
        complete_ts = pd.DataFrame({
            'ds': date_range,
            'unique_id': product_id
        })
        
        # Merge with actual data
        merged = pd.merge(complete_ts, group, on=['ds', 'unique_id'], how='left')
        
        # Enhanced interpolation strategy
        numeric_cols = ['y', 'qty', 'margin', 'gross_sales', 'discount']
        
        for col in numeric_cols:
            if col in merged.columns:
                if missing_ratio < 0.2:
                    merged[col] = merged[col].interpolate(method='linear')
                elif missing_ratio < 0.4:
                    merged[col] = merged[col].interpolate(method='linear', limit=5)
                else:
                    merged[col] = merged[col].interpolate(method='linear', limit=2)
                
                # Forward/backward fill for remaining gaps, then zero
                merged[col] = merged[col].fillna(method='ffill', limit=3)
                merged[col] = merged[col].fillna(method='bfill', limit=3)
                merged[col] = merged[col].fillna(0)
                merged[col] = merged[col].clip(lower=0)
        
        # Handle exogenous variables (forward fill since they're categorical/binary)
        exog_cols = [col for col in merged.columns if col not in ['ds', 'unique_id'] + numeric_cols]
        for col in exog_cols:
            if col in merged.columns:
                merged[col] = merged[col].fillna(method='ffill')
                merged[col] = merged[col].fillna(method='bfill')
                merged[col] = merged[col].fillna(0)
        
        # Verify frequency can be inferred
        freq_check = pd.infer_freq(merged['ds'])
        if freq_check == 'D':
            cleaned_data.append(merged)
            successful_products.append(product_id)
            print(f"        ✅ Success: {missing_points} gaps filled")
        else:
            print(f"        ❌ Frequency check failed")
    
    if cleaned_data:
        final_data = pd.concat(cleaned_data, ignore_index=True)
        print(f"   ✅ Robust processing complete: {len(successful_products)} products")
        print(f"   📊 Final dataset: {len(final_data):,} rows")
        return final_data
    else:
        raise ValueError("No products survived robust frequency correction")

# ========================================================================
# STEP 4: TRAIN-TEST SPLIT
# ========================================================================

def create_train_test_split(data, test_days=21):
    """Create train-test split"""
    print(f"   • Splitting data with {test_days} test days...")
    
    # Split data by taking last N days for testing
    test_data = data.groupby("unique_id").tail(test_days)
    train_data = data.groupby("unique_id").apply(lambda group: group.iloc[:-test_days]).reset_index(drop=True)
    
    print(f"   ✅ Train: {len(train_data)} rows, Test: {len(test_data)} rows")
    
    return train_data, test_data

# ========================================================================
# STEP 5: FUTURE EXOGENOUS DATA CREATION
# ========================================================================

def create_future_exogenous_data(train_data, exog_features, horizon):
    """Create future exogenous variable values for forecasting"""
    future_data = []
    
    for unique_id in train_data['unique_id'].unique():
        # Get last date for this product
        product_data = train_data[train_data['unique_id'] == unique_id]
        last_date = product_data['ds'].max()
        
        # Generate future dates
        future_dates = pd.date_range(
            start=last_date + pd.Timedelta(days=1),
            periods=horizon,
            freq='D'
        )
        
        for date in future_dates:
            row = {'unique_id': unique_id, 'ds': date}
            
            # Add predictable calendar features
            if 'year' in exog_features:
                row['year'] = date.year
            if 'month' in exog_features:
                row['month'] = date.month
            if 'day_of_week' in exog_features:
                row['day_of_week'] = date.dayofweek
            if 'is_weekend' in exog_features:
                row['is_weekend'] = int(date.dayofweek >= 5)
            if 'is_month_start' in exog_features:
                row['is_month_start'] = int(date.is_month_start)
            if 'is_month_end' in exog_features:
                row['is_month_end'] = int(date.is_month_end)
            if 'is_quarter_start' in exog_features:
                row['is_quarter_start'] = int(date.is_quarter_start)
            if 'is_quarter_end' in exog_features:
                row['is_quarter_end'] = int(date.is_quarter_end)
            if 'quarter' in exog_features:
                row['quarter'] = date.quarter
            
            # Add holiday indicators
            if 'is_republic_day' in exog_features:
                row['is_republic_day'] = int(date.strftime('%m-%d') == '01-26')
            if 'is_independence_day' in exog_features:
                row['is_independence_day'] = int(date.strftime('%m-%d') == '08-15')
            if 'is_christmas' in exog_features:
                row['is_christmas'] = int(date.strftime('%m-%d') == '12-25')
            
            # For variable date holidays, set to 0 for now (can be enhanced later)
            for holiday in ['is_holi', 'is_diwali']:
                if holiday in exog_features:
                    row[holiday] = 0
            
            future_data.append(row)
    
    return pd.DataFrame(future_data)

# ========================================================================
# STEP 6: TIMEGPT FORECASTING
# ========================================================================

def run_timegpt_forecasting_with_exogenous(train_data, predictable_features, test_periods=7):
    """TimeGPT forecasting with proper exogenous variable handling"""
    print(f"   🎯 Running TimeGPT forecast for {test_periods} periods...")
    
    try:
        # Check available exogenous features
        available_exog = [f for f in predictable_features if f in train_data.columns]
        
        if available_exog:
            print(f"   • Using exogenous variables: {available_exog}")
            
            # Create future exogenous data for the test period
            future_X_df = create_future_exogenous_data(train_data, available_exog, test_periods)
            
            # Main dataframe with target and historical exogenous variables
            main_data = train_data[['unique_id', 'ds', 'y'] + available_exog].copy()
            
            print(f"   • Main data shape: {main_data.shape}")
            print(f"   • Future exogenous data shape: {future_X_df.shape}")
            
            # Generate forecasts with exogenous variables
            forecasts = timegpt.forecast(
                df=main_data,          # Historical data with exogenous variables
                X_df=future_X_df,      # Future exogenous variables
                h=test_periods,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id',
                finetune_steps=30,
                finetune_loss='mae',
                clean_ex_first=True
            )
            
        else:
            print("   • No exogenous variables available, using target only")
            main_data = train_data[['unique_id', 'ds', 'y']].copy()
            
            forecasts = timegpt.forecast(
                df=main_data,
                h=test_periods,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id',
                finetune_steps=30,
                finetune_loss='mae'
            )
        
        print(f"   ✅ Forecasts generated: {len(forecasts):,} rows")
        return forecasts
        
    except Exception as e:
        print(f"   ❌ Forecasting failed: {e}")
        print("   🔄 Trying basic approach without exogenous variables...")
        
        # Fallback: Use only target variables
        try:
            basic_data = train_data[['unique_id', 'ds', 'y']].copy()
            
            forecasts = timegpt.forecast(
                df=basic_data,
                h=test_periods,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id',
                finetune_steps=20,
                finetune_loss='mae'
            )
            
            print(f"   ✅ Basic forecasts generated: {len(forecasts):,} rows")
            return forecasts
            
        except Exception as e2:
            print(f"   ❌ All approaches failed: {e2}")
            return None

# ========================================================================
# STEP 7: EVALUATION
# ========================================================================

def evaluate_forecasts(forecasts, test_data):
    """Evaluate forecast performance with improved MAPE handling"""
    if forecasts is None:
        return None
        
    print("   • Calculating performance metrics...")
    
    # Fix datetime types
    forecasts['ds'] = pd.to_datetime(forecasts['ds'])
    test_data['ds'] = pd.to_datetime(test_data['ds'])
    
    # Merge forecasts with test data
    evaluation_data = pd.merge(
        test_data,
        forecasts[['ds', 'unique_id', 'TimeGPT']],
        on=['ds', 'unique_id'],
        how='inner'
    )
    
    if len(evaluation_data) == 0:
        print("   ❌ No matching data for evaluation")
        return None
    
    # Calculate metrics by product
    product_metrics = []
    
    for product_id in evaluation_data['unique_id'].unique():
        product_eval = evaluation_data[evaluation_data['unique_id'] == product_id]
        
        if len(product_eval) > 0:
            # Standard metrics
            mse = mean_squared_error(product_eval['y'], product_eval['TimeGPT'])
            mae = mean_absolute_error(product_eval['y'], product_eval['TimeGPT'])
            rmse = np.sqrt(mse)
            
            # Improved MAPE calculation with zero handling
            actual_values = product_eval['y'].values
            predicted_values = product_eval['TimeGPT'].values
            
            # Exclude zero values from MAPE calculation
            non_zero_mask = actual_values != 0
            if non_zero_mask.sum() > 0:
                mape = np.mean(np.abs((actual_values[non_zero_mask] - predicted_values[non_zero_mask]) / actual_values[non_zero_mask])) * 100
                effective_points = non_zero_mask.sum()
            else:
                mape = np.inf
                effective_points = 0
            
            # sMAPE (Symmetric MAPE) - more robust to zeros
            smape = np.mean(2 * np.abs(actual_values - predicted_values) / 
                           (np.abs(actual_values) + np.abs(predicted_values) + 1e-8)) * 100
            
            # WAPE (Weighted Absolute Percentage Error)
            if actual_values.sum() != 0:
                wape = np.sum(np.abs(actual_values - predicted_values)) / np.sum(np.abs(actual_values)) * 100
            else:
                wape = np.inf
            
            product_metrics.append({
                'product': product_id,
                'mse': mse,
                'mae': mae,
                'rmse': rmse,
                'mape': mape,
                'smape': smape,
                'wape': wape,
                'data_points': len(product_eval),
                'effective_points': effective_points,
                'zero_values': (actual_values == 0).sum()
            })
    
    metrics_df = pd.DataFrame(product_metrics)
    
    # Calculate overall statistics
    finite_mape_mask = np.isfinite(metrics_df['mape'])
    
    if finite_mape_mask.sum() > 0:
        avg_mape = metrics_df.loc[finite_mape_mask, 'mape'].mean()
        mape_note = f"(calculated from {finite_mape_mask.sum()}/{len(metrics_df)} products)"
    else:
        avg_mape = np.inf
        mape_note = "(all products have zero values - use sMAPE instead)"
    
    overall_stats = {
        'avg_mse': metrics_df['mse'].mean(),
        'avg_mae': metrics_df['mae'].mean(),
        'avg_rmse': metrics_df['rmse'].mean(),
        'avg_mape': avg_mape,
        'avg_smape': metrics_df['smape'].mean(),
        'avg_wape': metrics_df[np.isfinite(metrics_df['wape'])]['wape'].mean(),
        'total_products': len(metrics_df),
        'total_points': len(evaluation_data),
        'products_with_zeros': (metrics_df['zero_values'] > 0).sum()
    }
    
    print(f"   ✅ Evaluation complete: {len(metrics_df)} products evaluated")
    print(f"   📊 Average MAE: {overall_stats['avg_mae']:.2f}")
    print(f"   📊 Average RMSE: {overall_stats['avg_rmse']:.2f}")
    print(f"   📊 Average sMAPE: {overall_stats['avg_smape']:.2f}% (recommended)")
    print(f"   📊 Average MAPE: {overall_stats['avg_mape']:.2f}% {mape_note}")
    
    return {
        'product_metrics': metrics_df,
        'overall_stats': overall_stats,
        'evaluation_data': evaluation_data
    }

# ========================================================================
# STEP 8: VISUALIZATION
# ========================================================================

def create_comprehensive_visualizations(full_data, forecasts, test_data, evaluation_results):
    """Create comprehensive visualizations"""
    print("   • Creating visualizations...")
    
    if evaluation_results is None or forecasts is None:
        print("   ❌ No evaluation data for visualization")
        return
    
    try:
        # Use TimeGPT's built-in plotting function
        timegpt.plot(
            test_data,
            forecasts,
            models=["TimeGPT"],
            level=[90],
            time_col="ds",
            target_col="y",
            id_col="unique_id",
            max_insample_length=60
        )
        
        print("   ✅ TimeGPT native plot created")
        
    except Exception as e:
        print(f"   ⚠️ TimeGPT plot failed: {e}")
        print("   🔄 Creating custom matplotlib visualization...")
        
        # Fallback to custom visualization
        evaluation_data = evaluation_results['evaluation_data']
        product_metrics = evaluation_results['product_metrics']
        
        # Get top 3 products for plotting
        top_products = product_metrics.nsmallest(3, 'smape')['product'].tolist()
        
        fig, axes = plt.subplots(len(top_products), 1, figsize=(15, 5*len(top_products)))
        if len(top_products) == 1:
            axes = [axes]
        
        for idx, product_id in enumerate(top_products):
            # Get data for this product
            product_train = full_data[full_data['unique_id'] == product_id]
            product_test = evaluation_data[evaluation_data['unique_id'] == product_id]
            product_pred = forecasts[forecasts['unique_id'] == product_id]
            
            # Get metrics
            metrics = product_metrics[product_metrics['product'] == product_id].iloc[0]
            
            # Plot training data (last 30 days)
            if len(product_test) > 0:
                test_start = product_test['ds'].min()
                train_plot = product_train[product_train['ds'] >= test_start - pd.Timedelta(days=30)]
                
                axes[idx].plot(train_plot['ds'], train_plot['y'], 'b-', linewidth=2, label='Training Data', alpha=0.7)
                axes[idx].plot(product_test['ds'], product_test['y'], 'go-', linewidth=3, markersize=8, label='Actual Test Data')
                axes[idx].plot(product_pred['ds'], product_pred['TimeGPT'], 'r--', linewidth=3, marker='s', markersize=8, label='TimeGPT Predictions')
                
                # Add confidence intervals if available
                if 'TimeGPT-lo-90' in product_pred.columns:
                    axes[idx].fill_between(product_pred['ds'], product_pred['TimeGPT-lo-90'], product_pred['TimeGPT-hi-90'], color='red', alpha=0.2, label='90% Confidence')
                
                # Formatting
                title = f'Product: {product_id[:50]}...\n'
                title += f'sMAPE: {metrics["smape"]:.1f}% | MAE: {metrics["mae"]:.1f}'
                
                axes[idx].set_title(title, fontsize=12, fontweight='bold')
                axes[idx].set_xlabel('Date')
                axes[idx].set_ylabel('Sales')
                axes[idx].legend()
                axes[idx].grid(True, alpha=0.3)
                
                # Format dates
                axes[idx].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
                plt.setp(axes[idx].xaxis.get_majorticklabels(), rotation=45)
        
        plt.suptitle('🤖 TimeGPT Forecasting Results', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print("   ✅ Custom visualization created")

# ========================================================================
# STEP 9: FUTURE FORECASTING
# ========================================================================

def generate_future_forecasts(data, predictable_features, horizon=14):
    """Generate future forecasts with proper exogenous variable handling"""
    print(f"   • Generating {horizon}-day future forecasts...")
    
    try:
        # Check if exogenous variables exist
        available_exog = [col for col in predictable_features if col in data.columns]
        
        if available_exog:
            print(f"   • Including exogenous features: {available_exog}")
            
            # Create future exogenous data
            future_X_df = create_future_exogenous_data(data, available_exog, horizon)
            
            # Main data for forecasting
            main_data = data[['unique_id', 'ds', 'y'] + available_exog].copy()
            
            future_forecasts = timegpt.forecast(
                df=main_data,
                X_df=future_X_df,
                h=horizon,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id'
            )
        else:
            # No exogenous variables
            future_forecasts = timegpt.forecast(
                df=data[['unique_id', 'ds', 'y']],
                h=horizon,
                freq='D',
                level=[80, 90],
                time_col='ds',
                target_col='y',
                id_col='unique_id'
            )
        
        # Create summary
        forecast_summary = future_forecasts.groupby('unique_id').agg({
            'TimeGPT': ['mean', 'sum']
        }).round(2)
        
        forecast_summary.columns = ['avg_daily', 'total_forecast']
        forecast_summary = forecast_summary.sort_values('total_forecast', ascending=False)
        
        print(f"   ✅ Future forecasts generated")
        print(f"   📊 Top 5 products by projected sales:")
        print(forecast_summary.head())
        
        return future_forecasts
        
    except Exception as e:
        print(f"   ❌ Future forecast error: {e}")
        return None

# ========================================================================
# MAIN PIPELINE FUNCTION
# ========================================================================

def complete_timegpt_pipeline_with_exogenous(df_sorted, top_n_products=8):
    """Complete end-to-end TimeGPT forecasting pipeline with exogenous features"""
    print("🚀 STARTING COMPLETE TIMEGPT PIPELINE WITH EXOGENOUS FEATURES")
    print("=" * 70)
    
    # Step 1: Enhanced Data Cleaning and Preparation with Exogenous Features
    print("\n📋 STEP 1: DATA CLEANING WITH EXOGENOUS FEATURES")
    cleaned_data, predictable_features = clean_and_prepare_data_with_exogenous(df_sorted, top_n_products)
    
    # Step 2: Fix Frequency Issues
    print("\n🔧 STEP 2: FIXING FREQUENCY ISSUES")
    frequency_fixed_data = fix_frequency_issues_complete(cleaned_data)
    
    # Step 3: Train-Test Split
    print("\n✂️ STEP 3: TRAIN-TEST SPLIT")
    train_data, test_data = create_train_test_split(frequency_fixed_data)
    
    # Step 4: TimeGPT Forecasting with Exogenous Features
    print("\n🤖 STEP 4: TIMEGPT FORECASTING WITH EXOGENOUS FEATURES")
    forecasts = run_timegpt_forecasting_with_exogenous(train_data, predictable_features, test_periods=7)
    
    # Step 5: Evaluation
    print("\n📊 STEP 5: MODEL EVALUATION")
    evaluation_results = evaluate_forecasts(forecasts, test_data)
    
    # Step 6: Visualization
    print("\n📈 STEP 6: CREATING VISUALIZATIONS")
    create_comprehensive_visualizations(frequency_fixed_data, forecasts, test_data, evaluation_results)
    
    # Step 7: Future Forecasting
    print("\n🔮 STEP 7: FUTURE FORECASTING")
    future_forecasts = generate_future_forecasts(frequency_fixed_data, predictable_features, horizon=14)
    
    return {
        'cleaned_data': frequency_fixed_data,
        'train_data': train_data,
        'test_data': test_data,
        'forecasts': forecasts,
        'evaluation': evaluation_results,
        'future_forecasts': future_forecasts,
        'exogenous_features': predictable_features
    }

# ========================================================================
# EXECUTION
# ========================================================================

# Load your data first
# df_sorted should be your sorted dataframe with BILLING_DATE sorted

# Run the complete pipeline
results = complete_timegpt_pipeline_with_exogenous(df_sorted, top_n_products=8)

print("\n🎉 PIPELINE COMPLETED SUCCESSFULLY!")
print("=" * 70)
print("📊 RESULTS SUMMARY:")
if results['evaluation']:
    stats = results['evaluation']['overall_stats']
    print(f"• Products forecasted: {stats['total_products']}")
    print(f"• Average MAE: {stats['avg_mae']:.2f}")
    print(f"• Average sMAPE: {stats['avg_smape']:.2f}%")
print("• Check the visualizations above for detailed results")


🚀 STARTING COMPLETE TIMEGPT PIPELINE WITH EXOGENOUS FEATURES

📋 STEP 1: DATA CLEANING WITH EXOGENOUS FEATURES
   • Creating exogenous features before data aggregation...
   • Creating exogenous features...
   • Created 14 predictable exogenous features
   • Found 12853 negative sales records
   📊 Product filtering with exogenous features:
      • Viable for forecasting: 381
      • Selected top: 8
   • Aggregating with enhanced exogenous variables...


INFO:nixtlats.nixtla_client:Validating inputs...
INFO:nixtlats.nixtla_client:Preprocessing dataframes...


   ✅ Enhanced data with exogenous features: 6,245 rows, 8 products
   📊 Exogenous features included: 14

🔧 STEP 2: FIXING FREQUENCY ISSUES
   • Applying robust frequency correction for irregular data...
      Processing: 04042 | MENS WEAR | SMART CASUALS | BOTT... (790 records)
        📊 Missing ratio: 0.25%
        ✅ Success: 2 gaps filled
      Processing: 04042 | MENS WEAR | SMART CASUALS | TOPS... (792 records)
        📊 Missing ratio: 0.00%
        ✅ Success: 0 gaps filled
      Processing: 04063 | MENS CASUAL | ACTIVE WEAR | BOTT... (791 records)
        📊 Missing ratio: 0.13%
        ✅ Success: 1 gaps filled
      Processing: 04063 | MENS CASUAL | ACTIVE WEAR | TOPS... (792 records)
        📊 Missing ratio: 0.00%
        ✅ Success: 0 gaps filled
      Processing: D699 | WOMENS WEAR | ETHNIC WEAR | TOPWE... (788 records)
        📊 Missing ratio: 0.51%
        ✅ Success: 4 gaps filled
      Processing: D699 | WOMENS WEAR | ETHNIC WEAR | TOPWE... (709 records)
        📊 Missing rat

INFO:nixtlats.nixtla_client:Using the following exogenous variables: year, month, day_of_week, is_weekend, is_month_start, is_month_end, is_quarter_start, is_quarter_end, quarter, is_republic_day, is_independence_day, is_christmas, is_holi, is_diwali
INFO:nixtlats.nixtla_client:Calling Forecast Endpoint...
INFO:nixtlats.nixtla_client:Validating inputs...
INFO:nixtlats.nixtla_client:Preprocessing dataframes...


   ✅ Forecasts generated: 56 rows

📊 STEP 5: MODEL EVALUATION
   • Calculating performance metrics...
   ✅ Evaluation complete: 8 products evaluated
   📊 Average MAE: 11952.41
   📊 Average RMSE: 13744.33
   📊 Average sMAPE: 100.08% (recommended)
   📊 Average MAPE: 115.44% (calculated from 8/8 products)

📈 STEP 6: CREATING VISUALIZATIONS
   • Creating visualizations...
   ✅ TimeGPT native plot created

🔮 STEP 7: FUTURE FORECASTING
   • Generating 14-day future forecasts...
   • Including exogenous features: ['year', 'month', 'day_of_week', 'is_weekend', 'is_month_start', 'is_month_end', 'is_quarter_start', 'is_quarter_end', 'quarter', 'is_republic_day', 'is_holi', 'is_diwali', 'is_independence_day', 'is_christmas']


INFO:nixtlats.nixtla_client:Using the following exogenous variables: year, month, day_of_week, is_weekend, is_month_start, is_month_end, is_quarter_start, is_quarter_end, quarter, is_republic_day, is_independence_day, is_christmas, is_holi, is_diwali
INFO:nixtlats.nixtla_client:Calling Forecast Endpoint...
INFO:nixtlats.nixtla_client:Attempt 1 failed...


   ✅ Future forecasts generated
   📊 Top 5 products by projected sales:
                                                    avg_daily  total_forecast
unique_id                                                                    
F085 | WOMENS WEAR | ETHNIC WEAR | TOPWEAR | KU...   94967.11      1329539.58
04042 | MENS WEAR | SMART CASUALS | TOPS | SHIRTS    49429.55       692013.77
04042 | MENS WEAR | SMART CASUALS | BOTTOMS | T...   20649.48       289092.73
04063 | MENS CASUAL | ACTIVE WEAR | BOTTOMS | T...   20093.73       281312.17
04063 | MENS CASUAL | ACTIVE WEAR | TOPS | T SH...   15752.59       220536.23

🎉 PIPELINE COMPLETED SUCCESSFULLY!
📊 RESULTS SUMMARY:
• Products forecasted: 8
• Average MAE: 11952.41
• Average sMAPE: 100.08%
• Check the visualizations above for detailed results
